## Remapping training data to the cubed sphere

- The novel addition in DLWP-CS is the ability to train convolutional neural networks on data mapped to the cubed sphere. 
- The re-mapping is performed offline from the model training/inference. 

#### Required packages

We use the TempestRemap library for cubed sphere remapping which is available as a pre-compiled conda package.

Let's use the DLWP CubeSphereRemap class on the data we processed earlier.

In [1]:
import os
import warnings
from DLWP.remap import CubeSphereRemap
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
#os.chdir (f"/scratch/vp91/{os.environ['USER']}/NCI-DLWP-CS")
#data_directory = f"/scratch/vp91/{os.environ['USER']}/NCI-DLWP-CS/Data/NCI_tutorial"
os.chdir (f"/tmp/{os.environ['USER']}/NCI-DLWP-CS")
data_directory = f"/tmp/{os.environ['USER']}/NCI-DLWP-CS/Data/NCI_tutorial"

processed_file = f'{data_directory}/NCI_tutorial_z500_t2m.nc' 
remapped_file  = f'{data_directory}/NCI_tutorial_z500_t2m_CS.nc' 
csr = CubeSphereRemap()

Generate the offline maps. 
- Since we used 2 degree data, we have 91 latitude points and 180 longitude points. We are mapping to a cubed sphere with 48 points on the side of each cube face. 
- Since data from CDS comes with monotonically decreasing latitudes, we specify the `inverse_lat` option. 
- New versions of TempestRemap have added the capability to read the coordinates from a netCDF file with any latitude/longitude coordinate names. (install it from source to use the `generate_offline_maps_from_file` method.)

In [2]:
csr.generate_offline_maps(lat=32, lon=64, res=48, inverse_lat=True)

CubeSphereRemap: generating offline forward map...
/opt/conda/envs/nci-dlwp-cs/bin/GenerateRLLMesh --lat 32 --lon 64 --file outLL.g --lat_begin 90 --lat_end -90 --out_format Netcdf4
/opt/conda/envs/nci-dlwp-cs/bin/GenerateCSMesh --res 48 --file outCS.g --out_format Netcdf4
/opt/conda/envs/nci-dlwp-cs/bin/GenerateOverlapMesh --a outLL.g --b outCS.g --out ov_LL_CS.g --out_format Netcdf4
/opt/conda/envs/nci-dlwp-cs/bin/GenerateOfflineMap --in_mesh outLL.g --out_mesh outCS.g --ov_mesh ov_LL_CS.g --in_np 1 --in_type FV --out_type FV --out_map map_LL32x64_CS48.nc --out_format Netcdf4
CubeSphereRemap: generating offline inverse map...
/opt/conda/envs/nci-dlwp-cs/bin/GenerateOverlapMesh --a outCS.g --b outLL.g --out ov_CS_LL.g --out_format Netcdf4
/opt/conda/envs/nci-dlwp-cs/bin/GenerateOfflineMap --in_mesh outCS.g --out_mesh outLL.g --ov_mesh ov_CS_LL.g --in_np 1 --in_type FV --out_type FV --out_map map_CS48_LL32x64.nc --out_format Netcdf4
CubeSphereRemap: successfully generated offline maps 

- Apply the forward map, saving to a temporary file. We specify to operate on the variable `predictors`, which is the only variable in the processed data. 
- TempestRemap is very finicky about metadata in netCDF files, sometimes failing with segmentation faults for no apparent reason. The most common crash is because it does not like the string coordinate values in the `'varlev'` coordinate. If you used the command in the previous tutorial to produce an extra "nocoord" version of this file, you might *have to* use it here.

In [3]:
csr.remap(processed_file + '.nocoord', '%s/temp.nc' % data_directory, '--var', 'predictors')

CubeSphereRemap: applying forward map...
/opt/conda/envs/nci-dlwp-cs/bin/ApplyOfflineMap --in_data /tmp/mah900/NCI-DLWP-CS/Data/NCI_tutorial/NCI_tutorial_z500_t2m.nc.nocoord --out_data /tmp/mah900/NCI-DLWP-CS/Data/NCI_tutorial/temp.nc --map map_LL32x64_CS48.nc --var predictors
CubeSphereRemap: successfully remapped data into /tmp/mah900/NCI-DLWP-CS/Data/NCI_tutorial/temp.nc


By default, TempestRemap has a 1-dimensional spatial coordinate. We convert the file to 3-dimensional faces (face, height, width). A few other points here:  
- Even if TempestRemap does not crash, it will probably delete the string coordinates, and sometimes the sample time coordinate as well, so it's a good idea to use this feature.  
- We also take advantage of the `chunking` parameter to save data with ideal chunking when using the file for training and evaluating models.

In [4]:
csr.convert_to_faces('%s/temp.nc' % data_directory, 
                     remapped_file,
                     coord_file=processed_file,
                     chunking={'sample': 1, 'varlev': 1})

CubeSphereRemap.convert_to_faces: loading data to memory...
CubeSphereRemap.convert_to_faces: assigning new coordinates to dataset
CubeSphereRemap.convert_to_faces: exporting data to file /tmp/mah900/NCI-DLWP-CS/Data/NCI_tutorial/NCI_tutorial_z500_t2m_CS.nc...
CubeSphereRemap.convert_to_faces: successfully exported reformatted data


<xarray.Dataset> Size: 2GB
Dimensions:     (face: 6, height: 48, width: 48, sample: 17528, varlev: 2)
Coordinates:
  * face        (face) int64 48B 0 1 2 3 4 5
  * height      (height) int64 384B 0 1 2 3 4 5 6 7 ... 40 41 42 43 44 45 46 47
  * width       (width) int64 384B 0 1 2 3 4 5 6 7 8 ... 40 41 42 43 44 45 46 47
  * sample      (sample) datetime64[ns] 140kB 2013-01-01 ... 2018-12-31T21:00:00
  * varlev      (varlev) <U5 40B 'z/500' 't2m/0'
Data variables:
    lon         (face, height, width) float64 111kB dask.array<chunksize=(6, 48, 48), meta=np.ndarray>
    lat         (face, height, width) float64 111kB dask.array<chunksize=(6, 48, 48), meta=np.ndarray>
    predictors  (sample, varlev, face, height, width) float32 2GB dask.array<chunksize=(1, 1, 6, 48, 48), meta=np.ndarray>
Attributes:
    description:  Training data for DLWP
    scaling:      True

In [5]:

os.remove('%s/temp.nc' % data_directory)